<a href="https://colab.research.google.com/github/barouk/proj/blob/main/custom_objectdetaction_in_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%rm -r yolov5
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="c1monuNXo97FVr52C3Q")
project = rf.workspace("zanajan-university").project("apple-adyti")
dataset = project.version(7).download("yolov5")#h




In [ ]:
#!python ./yolov5/train.py --img 640 --batch 16 --epochs 5 --data /content/apple-7/data.yaml



!python ./yolov5/train.py --img 640 --batch 16 --epochs 60 --data /content/apple-7/data.yaml --weights yolov5m.pt --cache



In [ ]:
import torch


model = torch.hub.load('ultralytics/yolov5', 'custom','/content/yolov5/runs/train/exp5/weights/best.pt')
im = '/content/apple.jpg'
results = model(im)
results.show()
print(results.xyxy[0])
print(results.pandas().xyxy[0] )
print(results.pandas().xyxy[0].value_counts('name'))


In [ ]:

!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt  --source /content/app.jpg

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# بارگیری مدل PT آموزش دیده شده
model = torch.load('yolo_model.pt')

# بارگیری داده‌های جدید و تولید برچسب متناظر با آن‌ها با استفاده از YOLO
new_images, yolo_labels = load_new_data()

# تبدیل داده‌ها به tensor
new_images_tensor = torch.tensor(new_images)
yolo_labels_tensor = torch.tensor(yolo_labels)

# تبدیل داده‌های جدید به شیء DataLoader
new_data = TensorDataset(new_images_tensor, yolo_labels_tensor)
new_data_loader = DataLoader(new_data, batch_size=32)

# اضافه کردن داده‌های جدید و برچسب‌های آن‌ها به داده‌های آموزشی قبلی
train_data = DataLoader(train_dataset, batch_size=32)
train_data = torch.utils.data.ConcatDataset([train_data.dataset, new_data])
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# تعریف متغیرهای ضروری برای آموزش مجدد مدل
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# آموزش مدل با داده‌های آموزشی جدید
for epoch in range(10):
    for images, labels in train_data_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# ذخیره مدل جدید
torch.save(model, 'updated_yolo_model.pt')